# W266 Final Project
## Ehsan Yousefzadeh 
## Summer 2018

In [1]:
from __future__ import division
from __future__ import print_function
import os, sys, re, json, time, datetime, shutil
import itertools, collections
from importlib import reload
from IPython.display import display, HTML

# NLTK 
import nltk

# NumPy, Pandas and TensorFlow
import numpy as np
import pandas as pd
import tensorflow as tf
assert(tf.__version__.startswith("1.8"))
import collections

#SKlearn 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Helper libraries
from w266_common import utils, vocabulary, tf_embed_viz, treeviz
from w266_common import patched_numpy_io
# Code for this assignment
import sst, models, models_test

# Monkey-patch NLTK with better Tree display that works on Cloud or other display-less server.
print("Overriding nltk.tree.Tree pretty-printing to use custom GraphViz.")
treeviz.monkey_patch(nltk.tree.Tree, node_style_fn=sst.sst_node_style, format='svg')

/home/ehsanyousefzadeh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Overriding nltk.tree.Tree pretty-printing to use custom GraphViz.


# Data Loading and Preparation 

In [2]:
scotch_data = pd.read_csv('./scotch_review.csv')

In [3]:
scotch_data

,Unnamed: 0,name,category,review.point,price,currency,description
0,1,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,97,225,$,"Magnificently powerful and intense. Caramels, ..."
1,2,"Black Bowmore, 1964 vintage, 42 year old, 40.5%",Single Malt Scotch,97,4500.00,$,What impresses me most is how this whisky evol...
2,3,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,97,13500.00,$,There have been some legendary Bowmores from t...
3,4,"Compass Box The General, 53.4%",Blended Malt Scotch Whisky,96,325,$,With a name inspired by a 1926 Buster Keaton m...
4,5,"Chivas Regal Ultis, 40%",Blended Malt Scotch Whisky,96,160,$,"Captivating, enticing, and wonderfully charmin..."
5,6,"Ardbeg Corryvreckan, 57.1%",Single Malt Scotch,96,85.00,$,"Powerful, muscular, well-textured, and invigor..."
6,7,"Gold Bowmore, 1964 vintage, 42.4%",Single Malt Scotch,96,6250.00,$,Deep gold color. Surprisingly lively on the no...
7,8,"Bowmore, 40 year old, 44.8%",Single Malt Scotch,96,11000.00,$,"Definitely showing its age, but not in a bad w..."
8,9,"The Dalmore, 50 year old, 52.8%",Single Malt Scotch,96,1500.00,$,The Dalmore is one of a handful of whiskies th...
9,10,"Glenfarclas Family Casks 1954 Cask #1260, 47.2%",Single Malt Scotch,96,3360,$,A rich amber color and elegantly oxidized note...


In [4]:
scotch_data = scotch_data.drop(scotch_data.index[19])

In [5]:
scotch_data = scotch_data[scotch_data.price != '60,000/set']

In [6]:
scotch_data = scotch_data[scotch_data.price != '44/liter']

In [7]:
scotch_y = scotch_data.price.replace({',':''},regex=True).apply(pd.to_numeric,1)

In [8]:
scotch_y = scotch_y.as_matrix()

In [9]:
scotch_y = scotch_y.astype(float)

In [10]:
np.mean(scotch_y)

531.0312360553324

In [11]:
scotch_y1 = []

for i in scotch_y:

    if i >= np.median(scotch_y):
        y = 1
    if i< np.median(scotch_y): 
        y = 0
    scotch_y1.append(y)
    
    

In [12]:
scotch_y1

[1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,


In [13]:
scotch_x = scotch_data['description'].as_matrix()

In [14]:
scotch_x[5]

'Powerful, muscular, well-textured, and invigorating. Even within the realm of Ardbeg, this one stands out. The more aggressive notes of coal tar, damp kiln, anise, and smoked seaweed are supported by an array of fruit (black raspberry, black cherry, plum), dark chocolate, espresso, molasses, bacon fat, kalamata olive, and warming cinnamon on the finish. Quite stunning!'

In [15]:
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()


scotch_x_tokens = []

for x in scotch_x:
    

    tokens = tokenizer.tokenize(x)
    scotch_x_tokens.append(tokens)


In [16]:
scotch_x_tokens[5]

['Powerful',
 ',',
 'muscular',
 ',',
 'well-textured',
 ',',
 'and',
 'invigorating.',
 'Even',
 'within',
 'the',
 'realm',
 'of',
 'Ardbeg',
 ',',
 'this',
 'one',
 'stands',
 'out.',
 'The',
 'more',
 'aggressive',
 'notes',
 'of',
 'coal',
 'tar',
 ',',
 'damp',
 'kiln',
 ',',
 'anise',
 ',',
 'and',
 'smoked',
 'seaweed',
 'are',
 'supported',
 'by',
 'an',
 'array',
 'of',
 'fruit',
 '(',
 'black',
 'raspberry',
 ',',
 'black',
 'cherry',
 ',',
 'plum',
 ')',
 ',',
 'dark',
 'chocolate',
 ',',
 'espresso',
 ',',
 'molasses',
 ',',
 'bacon',
 'fat',
 ',',
 'kalamata',
 'olive',
 ',',
 'and',
 'warming',
 'cinnamon',
 'on',
 'the',
 'finish.',
 'Quite',
 'stunning',
 '!']

In [17]:
scotch_x_canon = []


for x in scotch_x_tokens: 
    canon = utils.canonicalize_words(x)
    scotch_x_canon.append(canon)


In [18]:
scotch_x_canon[5]

['powerful',
 ',',
 'muscular',
 ',',
 'well-textured',
 ',',
 'and',
 'invigorating.',
 'even',
 'within',
 'the',
 'realm',
 'of',
 'ardbeg',
 ',',
 'this',
 'one',
 'stands',
 'out.',
 'the',
 'more',
 'aggressive',
 'notes',
 'of',
 'coal',
 'tar',
 ',',
 'damp',
 'kiln',
 ',',
 'anise',
 ',',
 'and',
 'smoked',
 'seaweed',
 'are',
 'supported',
 'by',
 'an',
 'array',
 'of',
 'fruit',
 '(',
 'black',
 'raspberry',
 ',',
 'black',
 'cherry',
 ',',
 'plum',
 ')',
 ',',
 'dark',
 'chocolate',
 ',',
 'espresso',
 ',',
 'molasses',
 ',',
 'bacon',
 'fat',
 ',',
 'kalamata',
 'olive',
 ',',
 'and',
 'warming',
 'cinnamon',
 'on',
 'the',
 'finish.',
 'quite',
 'stunning',
 '!']

In [19]:
scotch_flat = []
for x in scotch_x_canon:
    for i in x:
        scotch_flat.append(i)

In [20]:
scotch_flat

['magnificently',
 'powerful',
 'and',
 'intense.',
 'caramels',
 ',',
 'dried',
 'peats',
 ',',
 'elegant',
 'cigar',
 'smoke',
 ',',
 'seeds',
 'scraped',
 'from',
 'vanilla',
 'beans',
 ',',
 'brand',
 'new',
 'pencils',
 ',',
 'peppercorn',
 ',',
 'coriander',
 'seeds',
 ',',
 'and',
 'star',
 'anise',
 'make',
 'for',
 'a',
 'deeply',
 'satisfying',
 'nosing',
 'experience.',
 'silky',
 'caramels',
 ',',
 'bountiful',
 'fruits',
 'of',
 'ripe',
 'peach',
 ',',
 'stewed',
 'apple',
 ',',
 'orange',
 'pith',
 ',',
 'and',
 'pervasive',
 'smoke',
 'with',
 'elements',
 'of',
 'burnt',
 'tobacco.',
 'an',
 'abiding',
 'finish',
 'of',
 'smoke',
 ',',
 'dry',
 'spices',
 ',',
 'and',
 'banoffee',
 'pie',
 'sweetness.',
 'close',
 'to',
 'perfection.',
 'editor',
 "'s",
 'choice',
 'what',
 'impresses',
 'me',
 'most',
 'is',
 'how',
 'this',
 'whisky',
 'evolves',
 ';',
 'it',
 "'s",
 'incredibly',
 'complex.',
 'on',
 'the',
 'nose',
 'and',
 'palate',
 ',',
 'this',
 'is',
 'a',
 'th

In [21]:
vocab = vocabulary.Vocabulary(scotch_flat, size=None)  # size=None means unlimited
print("Vocabulary size: {:,}".format(vocab.size))


scotch_x_ids = []
for x in scotch_x_canon:
    x_ids = vocab.words_to_ids(x)
    scotch_x_ids.append(x_ids)

Vocabulary size: 11,520


In [22]:
vocabulary = vocab.size
vocabulary

11520

In [23]:
scotch_x_ids[5]

[612,
 3,
 1981,
 3,
 4130,
 3,
 4,
 2833,
 191,
 1800,
 5,
 2834,
 7,
 554,
 3,
 11,
 72,
 3353,
 1226,
 5,
 32,
 1227,
 22,
 7,
 555,
 363,
 3,
 252,
 802,
 3,
 406,
 3,
 4,
 325,
 246,
 50,
 4131,
 59,
 37,
 988,
 7,
 30,
 17,
 96,
 350,
 3,
 96,
 292,
 3,
 339,
 18,
 3,
 89,
 47,
 3,
 894,
 3,
 584,
 3,
 1463,
 949,
 3,
 2173,
 614,
 3,
 4,
 420,
 107,
 13,
 5,
 48,
 85,
 1531,
 154]

In [24]:
max_len = 300
scotch_x_pad = []
scotch_ns = []
for i in scotch_x_ids:
    
    pad_len = max_len - len(i)
    pad_i = np.pad(i, (0, pad_len), 'constant', constant_values =0)
    scotch_ns.append(len(i))
    scotch_x_pad.append(pad_i)

In [25]:
scotch_x_pad[5]

array([ 612,    3, 1981,    3, 4130,    3,    4, 2833,  191, 1800,    5,
       2834,    7,  554,    3,   11,   72, 3353, 1226,    5,   32, 1227,
         22,    7,  555,  363,    3,  252,  802,    3,  406,    3,    4,
        325,  246,   50, 4131,   59,   37,  988,    7,   30,   17,   96,
        350,    3,   96,  292,    3,  339,   18,    3,   89,   47,    3,
        894,    3,  584,    3, 1463,  949,    3, 2173,  614,    3,    4,
        420,  107,   13,    5,   48,   85, 1531,  154,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [26]:
scotch_x_pad_np = np.asarray(scotch_x_pad)
scotch_y_np = np.asarray(scotch_y)
# scotch_y_np = scotch_y_np.dtype(np.float32)
scotch_ns_np = np.asarray(scotch_ns)

In [27]:
train_x = scotch_x_pad_np[0:1790]
train_ns = scotch_ns_np[0:1790]
train_y = scotch_y_np[0:1790]

test_x = scotch_x_pad_np[1791:2100]
test_ns = scotch_ns_np[1791:2100]
test_y = scotch_y_np[1791:2100]

dev_x = scotch_x_pad_np[2101::]
dev_ns = scotch_ns_np[2101::]
dev_y = scotch_y_np[2101::]


In [28]:
import sst
ds = sst.SSTDataset(V=20000).process(label_scheme="binary")

Loading SST from data/sst/trainDevTestTrees_PTB.zip
Training set:     8,544 trees
Development set:  1,101 trees
Test set:         2,210 trees
Building vocabulary - 16,474 words
Processing to phrases...  Done!
Splits: train / dev / test : 98,794 / 13,142 / 26,052


In [29]:
test_y

array([  85.,   90.,  113.,  690.,  110.,  120.,   55.,  170.,  170.,
        180.,   60.,   50.,   85.,   88.,  117.,  138.,   84.,   85.,
        150.,   45.,  265.,  135.,  140.,  155.,  160.,  100.,   73.,
        160.,  210.,  592.,  125.,   36.,   90.,  155.,  900.,   50.,
        100.,  156.,  700.,   89.,   67.,   73.,  440.,   51.,   90.,
         93.,  111.,  130.,   45.,   49.,  220.,   80.,  137.,   76.,
         75.,   65.,  121.,  190.,   70.,   44.,   90., 2750.,   70.,
         56.,  567.,  185.,   63.,   97.,   70.,   40.,   40.,   73.,
         55.,  180.,   70.,   65.,   30.,   70.,  119.,  154.,  164.,
        104.,  190.,   54.,  125.,   60.,  108.,   78.,   40.,  325.,
        100.,   90.,   45.,   61.,   61.,   46.,  119.,  160.,  100.,
        142.,   77.,   32.,   98.,   60.,  168.,  119.,   73.,   69.,
        110.,   90.,   90.,   95.,   72.,   73.,   76.,   37.,   33.,
         44.,   53.,   65.,  348.,   61.,  115.,  126.,   60.,  195.,
         69.,   22.,

# Base Model (Naive Bayes) Training and Evaluation

In [30]:
nb = MultinomialNB()

nb.fit(train_x, train_y)
y_pred = nb.predict(test_x)

acc = accuracy_score(test_y, y_pred)

print("Accuracy: {:.02f}%".format(100*acc))

Accuracy: 0.65%


In [31]:
y_pred

array([6.900e+01, 1.070e+02, 3.200e+01, 9.500e+01, 3.000e+01, 1.080e+02,
       7.800e+01, 7.800e+01, 2.250e+02, 6.700e+01, 3.000e+04, 1.510e+02,
       1.510e+02, 2.500e+02, 1.100e+03, 7.000e+01, 3.500e+03, 1.100e+02,
       9.900e+01, 2.500e+02, 4.060e+02, 8.000e+01, 4.000e+02, 2.000e+02,
       5.000e+01, 1.190e+02, 1.510e+02, 4.150e+02, 2.000e+02, 8.500e+01,
       1.170e+02, 1.140e+02, 5.000e+01, 6.700e+01, 4.220e+02, 1.080e+02,
       5.000e+01, 1.500e+03, 4.500e+03, 8.000e+01, 9.500e+01, 9.900e+01,
       9.900e+01, 2.400e+02, 1.010e+02, 2.250e+02, 1.900e+03, 1.450e+02,
       2.500e+01, 8.000e+01, 2.940e+02, 1.250e+02, 6.300e+01, 9.700e+01,
       6.000e+02, 7.800e+01, 1.150e+02, 9.900e+01, 1.230e+02, 1.510e+02,
       1.380e+02, 8.000e+01, 7.200e+01, 1.150e+02, 3.000e+02, 4.500e+01,
       7.500e+01, 3.000e+01, 8.200e+01, 5.120e+02, 1.230e+02, 9.600e+01,
       2.500e+02, 6.300e+01, 9.900e+01, 8.900e+01, 4.800e+01, 1.030e+02,
       4.800e+01, 3.000e+02, 1.150e+02, 6.700e+01, 

# Neural Network Model Training and Evaluation 

In [32]:
from __future__ import print_function
from __future__ import division

import tensorflow as tf

def embedding_layer(ids_, V, embed_dim, init_scale=0.001):
    
    W_embed_ = tf.get_variable(name = "W_embed", shape = [V, embed_dim],
    initializer = tf.random_uniform_initializer( minval = -init_scale, maxval = init_scale))                 
    xs_ = tf.nn.embedding_lookup(W_embed_, ids_)

    return xs_

def fully_connected_layers(h0_, hidden_dims, activation=tf.tanh,
                           dropout_rate=0, is_training=False):
    h_ = h0_
    for i, hdim in enumerate(hidden_dims):
        h_ = tf.layers.dense(h_, hdim, activation=activation, name=("Hidden_%d"%i))

        if dropout_rate > 0:
            h_ =  tf.layers.dropout(h_, rate=dropout_rate)

    return h_

def output_layer(h_, labels_, num_classes):
    with tf.variable_scope("Logits"):

        W_out_ = tf.get_variable(name = "W_out", shape=[h_.shape[1], num_classes], initializer = tf.random_normal_initializer())
        b_out_ = tf.get_variable("b_out", shape = [num_classes], initializer=tf.zeros_initializer())
        logits_ = tf.add(tf.matmul(h_, W_out_), b_out_) 
        predictions_ = tf.squeeze(logits_, 1)

        
    if labels_ is None:
        return None, logits_, W_out_

    with tf.name_scope("RMSE"):
        loss_ = tf.losses.mean_squared_error(labels = labels_, predictions = predictions_)

        
    return loss_, logits_, W_out_



def BOW_encoder(ids_, ns_, V, embed_dim, hidden_dims, dropout_rate=0,
                is_training=None,
                **unused_kw):
    assert is_training is not None, "is_training must be explicitly set to True or False"
 
    with tf.variable_scope("Embedding_Layer"):
        xs_ = embedding_layer(ids_, V, embed_dim, init_scale=0.001)

    mask_ = tf.expand_dims(tf.sequence_mask(ns_, xs_.shape[1], dtype=tf.float32),-1)

    sum_e = tf.reduce_sum(xs_*mask_, 1)

    h_ = fully_connected_layers(sum_e, hidden_dims, activation=tf.tanh,
            dropout_rate=dropout_rate, is_training=is_training)

    return h_, xs_


def classifier_model_fn(features, labels, mode, params):
    tf.set_random_seed(params.get('rseed', 10))

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    if params['encoder_type'] == 'bow':
        with tf.variable_scope("Encoder"):
            h_, xs_ = BOW_encoder(features['ids'], features['ns'],
                                  is_training=is_training,
                                  **params)
    else:
        raise ValueError("Error: unsupported encoder type "
                         "'{:s}'".format(params['encoder_type']))

    with tf.variable_scope("Output_Layer"):
        ce_loss_, logits_, W_out_ = output_layer(h_, labels ,params['num_classes'] )

    with tf.name_scope("Prediction"):
        
        pred_max_ = tf.squeeze(logits_, 1)
        
        predictions_dict = {"proba": logits_, "max": pred_max_}

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=predictions_dict)

    with tf.variable_scope("Regularization"):
        l2_penalty_ = tf.nn.l2_loss(xs_)  
        for var_ in tf.trainable_variables():
            if "Embedding_Layer" in var_.name:
                continue
            l2_penalty_ += tf.nn.l2_loss(var_)
        l2_penalty_ *= params['beta']  
        tf.summary.scalar("l2_penalty", l2_penalty_)
        regularized_loss_ = ce_loss_ + l2_penalty_

    with tf.variable_scope("Training"):
        if params['optimizer'] == 'adam':
            optimizer_ = tf.train.AdamOptimizer(params['lr'])
        else:
            optimizer_ = tf.train.GradientDescentOptimizer(params['lr'])
        train_op_ = optimizer_.minimize(ce_loss_,
                                            global_step=tf.train.get_global_step())

    tf.summary.scalar("cross_entropy_loss", ce_loss_)
    eval_metrics = {"cross_entropy_loss": tf.metrics.mean(ce_loss_),
                    "accuracy": tf.metrics.accuracy(labels, pred_max_)}

    return tf.estimator.EstimatorSpec(mode=mode,
                                      predictions=predictions_dict,
                                      loss=regularized_loss_,
                                      train_op=train_op_,
                                      eval_metric_ops=eval_metrics)


In [33]:

model_params = dict(V=vocabulary, embed_dim=50, hidden_dims=[25], num_classes = 1,
                    encoder_type='bow',
                    lr=0.1, optimizer='adam', beta=0.01)

checkpoint_dir = "/tmp/tf_bow_sst_" + datetime.datetime.now().strftime("%Y%m%d-%H%M")
if os.path.isdir(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)

ds.vocab.write_projector_config(checkpoint_dir, "Encoder/Embedding_Layer/W_embed")

model = tf.estimator.Estimator(model_fn= classifier_model_fn, 
                               params=model_params,
                               model_dir=checkpoint_dir)
print("")
print("To view training (once it starts), run:\n")
print("    tensorboard --logdir='{:s}' --port 6006".format(checkpoint_dir))
print("\nThen in your browser, open: http://localhost:6006")

Vocabulary (16,474 words) written to '/tmp/tf_bow_sst_20180807-2337/metadata.tsv'
Projector config written to /tmp/tf_bow_sst_20180807-2337/projector_config.pbtxt
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tf_bow_sst_20180807-2337', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f98604269b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}

To view training (once it starts), run:

    tensorboard --logdir='/tmp/tf_bow_sst_20180807-2337' --port 6006

Then in your browser, open: http://localhost:6006


In [34]:
train_params = dict(batch_size=32, total_epochs=20, eval_every=2)
assert(train_params['total_epochs'] % train_params['eval_every'] == 0)

train_input_fn = patched_numpy_io.numpy_input_fn(
                    x={"ids": train_x, "ns": train_ns}, y=train_y,
                    batch_size=train_params['batch_size'], 
                    num_epochs=train_params['eval_every'], shuffle=True, seed=42
                 )


dev_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": dev_x, "ns": dev_ns}, y=dev_y,
                    batch_size=128, num_epochs=1, shuffle=False
                )

for _ in range(train_params['total_epochs'] // train_params['eval_every']):
    model.train(input_fn=train_input_fn)
    eval_metrics = model.evaluate(input_fn=dev_input_fn, name="dev")

INFO:tensorflow:Calling model_fn.
Predictions Tensor("Output_Layer/Logits/Squeeze:0", shape=(?,), dtype=float32)
Logits:  Tensor("Output_Layer/Logits/Add:0", shape=(?, 1), dtype=float32)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tf_bow_sst_20180807-2337/model.ckpt.
INFO:tensorflow:loss = 1960503.8, step = 1
INFO:tensorflow:global_step/sec: 100.13
INFO:tensorflow:loss = 900913.9, step = 101 (1.000 sec)
INFO:tensorflow:Saving checkpoints for 112 into /tmp/tf_bow_sst_20180807-2337/model.ckpt.
INFO:tensorflow:Loss for final step: 860525.56.
INFO:tensorflow:Calling model_fn.
Predictions Tensor("Output_Layer/Logits/Squeeze:0", shape=(?,), dtype=float32)
Logits:  Tensor("Output_Layer/Logits/Add:0", shape=(?, 1), dtype=float32)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Sta

INFO:tensorflow:loss = 853677.75, step = 661 (0.948 sec)
INFO:tensorflow:Saving checkpoints for 672 into /tmp/tf_bow_sst_20180807-2337/model.ckpt.
INFO:tensorflow:Loss for final step: 800945.94.
INFO:tensorflow:Calling model_fn.
Predictions Tensor("Output_Layer/Logits/Squeeze:0", shape=(?,), dtype=float32)
Logits:  Tensor("Output_Layer/Logits/Add:0", shape=(?, 1), dtype=float32)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-07-23:38:03
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20180807-2337/model.ckpt-672
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-07-23:38:03
INFO:tensorflow:Saving dict for global step 672: accuracy = 0.0, cross_entropy_loss = 5713754.0, global_step = 672, loss = 5714213.5
INFO:tensorflow:Calling model_fn.
Predictions Tensor("Output_Layer/Logits/Squeeze:0", shape=(?,), dtype=float32)
Logits: 

In [35]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": test_x, "ns": test_ns}, y=test_y,
                    batch_size=128, num_epochs=1, shuffle=False)


eval_metrics = model.evaluate(input_fn=test_input_fn, name="test") 
eval_metrics

INFO:tensorflow:Calling model_fn.
Predictions Tensor("Output_Layer/Logits/Squeeze:0", shape=(?,), dtype=float32)
Logits:  Tensor("Output_Layer/Logits/Add:0", shape=(?, 1), dtype=float32)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-07-23:38:14
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20180807-2337/model.ckpt-1120
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-07-23:38:14
INFO:tensorflow:Saving dict for global step 1120: accuracy = 0.0, cross_entropy_loss = 276659.16, global_step = 1120, loss = 277314.16
Accuracy on test set: 0.00%


{'accuracy': 0.0,
 'cross_entropy_loss': 276659.16,
 'global_step': 1120,
 'loss': 277314.16}

In [36]:
from sklearn.metrics import accuracy_score
predictions = list(model.predict(test_input_fn)) 
y_pred = [p['max'] for p in predictions]

INFO:tensorflow:Calling model_fn.
Predictions Tensor("Output_Layer/Logits/Squeeze:0", shape=(?,), dtype=float32)
Logits:  Tensor("Output_Layer/Logits/Add:0", shape=(?, 1), dtype=float32)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20180807-2337/model.ckpt-1120
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [37]:
test_y

array([  85.,   90.,  113.,  690.,  110.,  120.,   55.,  170.,  170.,
        180.,   60.,   50.,   85.,   88.,  117.,  138.,   84.,   85.,
        150.,   45.,  265.,  135.,  140.,  155.,  160.,  100.,   73.,
        160.,  210.,  592.,  125.,   36.,   90.,  155.,  900.,   50.,
        100.,  156.,  700.,   89.,   67.,   73.,  440.,   51.,   90.,
         93.,  111.,  130.,   45.,   49.,  220.,   80.,  137.,   76.,
         75.,   65.,  121.,  190.,   70.,   44.,   90., 2750.,   70.,
         56.,  567.,  185.,   63.,   97.,   70.,   40.,   40.,   73.,
         55.,  180.,   70.,   65.,   30.,   70.,  119.,  154.,  164.,
        104.,  190.,   54.,  125.,   60.,  108.,   78.,   40.,  325.,
        100.,   90.,   45.,   61.,   61.,   46.,  119.,  160.,  100.,
        142.,   77.,   32.,   98.,   60.,  168.,  119.,   73.,   69.,
        110.,   90.,   90.,   95.,   72.,   73.,   76.,   37.,   33.,
         44.,   53.,   65.,  348.,   61.,  115.,  126.,   60.,  195.,
         69.,   22.,

In [38]:
y_pred

[608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,
 608.3514,